Import the libraries for the script

In [3]:
import findspark

findspark.init()
import pyspark
import pyspark.sql
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.mllib.stat import Statistics

spark = SparkSession.builder.getOrCreate()
SparkConf().getAll()

[(u'spark.master', u'local[*]'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.app.name', u'pyspark-shell')]

Import the data from local csv files

In [4]:
media = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load(path='C:\Users\ChrisGomes\Projects\MediaStock\gdelt\*.csv')
sp500 = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load(path='C:\Users\ChrisGomes\Projects\MediaStock\s&p500.csv')
sp500Companies = spark.read \
    .format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .load(path='C:\Users\ChrisGomes\Projects\MediaStock\s&p500Companies.csv')


Check correlation between average GoldsteinScale and price change per day

In [23]:
# get avg goldstein scale per date
goldsteinScales = media.groupby('SQLDATE').agg(pyspark.sql.functions.avg('GoldsteinScale'))


In [30]:
# calculate absolute price change
sp500 = sp500.withColumn('PriceChange', pyspark.sql.functions.abs(sp500['Open'] - sp500['Close']))

# Convert Date to SQLDATE format
sp500 = sp500.withColumn('SQLDATEString', pyspark.sql.functions.date_format('Date', 'yyyyMMdd'))

# Convert SQLDATE to int
sp500 = sp500.withColumn('SQLDATE', sp500['SQLDATEString'].cast('Int'))

# get PriceChange per date
priceChanges = sp500.groupby('SQLDATE').agg(pyspark.sql.functions.avg('PriceChange'))

In [38]:
# join the two measurements
goldsteinAndPrice = goldsteinScales.join(priceChanges, 'SQLDATE')

In [41]:
# find correlation
Statistics.corr(goldsteinAndPrice.drop('SQLDATE').collect(), method='pearson')